In [9]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import PyPDF2
import os


#### Collect the links on the page

In [10]:
url = "https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/"
response = requests.get(url + "/?showSvar=true&term=&isFilterOpen=true&page=3")

html_doc = response.text

soup = bs(html_doc, 'html.parser')

# Find the ul class that contains the articles
articles = soup.find("ul", class_="link-list")

# Create a dictionary to store the sites and pdfs
sites = {}
pdfs = {}

# Iterate through every li tag in the ul class
for article in articles.find_all("li"):
    if article.text.find("PDF") != -1:
        # Removes the (PDF) from the name of the actor (the writer of the article)
        actor = article.text.split("(")[0][:-1]
        pdfs[actor] = "https://www.regjeringen.no/" + article.find("a")["href"]
        continue
    sites[article.text] = url + article.find("a")["href"]


print(len(sites))
print(len(pdfs))


51
9


#### Read the htmls and preprocess them

In [11]:
df_texts = pd.DataFrame(columns=['actor', 'text', 'date'])

# Read the html files and extract the text

def preprocess_html(texts : list):
    # Remove the html tags
    paragraphs = [paragraph.text.strip() for paragraph in texts if paragraph.text != ""]

    # Remove the header "Høringssvar til NOU 2023: ...."
    paragraphs = [sections for sections in paragraphs if sections.find("Høringssvar til NOU 2023") == -1]

    return paragraphs




for actor, site in sites.items():
    # Remove the spaces at the end of the actor
    actor = actor.strip()
    print(actor.__repr__())
    response = requests.get(site)
    html_doc = response.text
    soup = bs(html_doc, 'html.parser')
    # Find the text of the article
    texts = soup.find("div", class_="article-body").find_all("p")
    text = preprocess_html(texts)

    # Find the date of the article
    date = soup.get_text().split("Dato")[1].split("\n")[0].split(" ")[1]

    # Make one str of the text
    text = " ".join(text)
    #for paragraph in text:
    df_texts.loc[len(df_texts)] = [actor, text, date]
    


print(actor)
df_texts
df_texts.to_csv("../../dataset/html_hearings_page3.csv", index=False)


'Sivilingeniør Arve Gleissner Gustavsen'
'Skagerak Energi'
'Skattedirektoratet (uten merknader)'
'Skift - Næringslivets Klimaledere'
'Skognæringa Kyst SA'
'Småkraftforeninga'
'SoCentral AS'
'Sokkeldirektoratet'
'Sola INP'
'Spire'
'Standard Norge'
'Statens Arbeidsmiljøinstitutt'
'Statistisk sentralbyrå'
'Statkraft AS'
'Statsbygg'
'Statsforvaltaren i Vestland'
'Statsforvalteren i Vestfold og Telemark'
'Stavanger kommune'
'Steinar Eikemo'
'Stig Eidesmo'
'Stord Industri og Næringsparti (Stord INP)'
'Sunnfjord kommune'
'Syklistforeningen'
'Tekna - Teknisk-naturvitenskapelig forening'
'Telemark fylkeskommune'
'Thom'
'Treindustrien'
'Troms fylkeskommune'
'Tromsø kommune'
'Trondheim kommune'
'Trysil kommune'
'Tysvær Industri- og Næringspartiet'
'Tønsberg kommune'
'Unio'
'Universitetet i Stavanger'
'Universitets- og høgskolerådet'
'Vaksdal INP'
'Vardø kommune'
'Vennesla kommune'
'Vestlandsforsking'
'Vestnes Industri- og Næringsparti (INP)'
'Veterinærinstituttet'
'Veterinært bærekraftforum'
'Vit

In [12]:
df_texts

,actor,text,date
0,Sivilingeniør Arve Gleissner Gustavsen,Høringsutvalgets rapporten fra 25. oktober 202...,03.02.2024
1,Skagerak Energi,Det vises til høring - NOU 2023:25 omstilling ...,28.02.2024
2,Skattedirektoratet (uten merknader),,27.02.2024
3,Skift - Næringslivets Klimaledere,Høringsinnspill på vegne av Skift – Næringsliv...,30.01.2024
4,Skognæringa Kyst SA,Norge bør ta sin del av ansvaret for skogplant...,29.01.2024
5,Småkraftforeninga,Vedlagt ligger Småkraftforeningas høringsuttal...,08.02.2024
6,SoCentral AS,SoCentral mener Klimautvalget har gjort et gru...,26.01.2024
7,Sokkeldirektoratet,Sokkeldirektoratet har hatt NOU 2023:25 på int...,27.02.2024
8,Sola INP,1. Bergen INP mener at skogen må regnes med i ...,25.02.2024
9,Spire,Spire ønsker å takke klimautvalget 2050 for en...,28.02.2024
